# Problem 2

## Set up parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Model Parameters
gamma = 0.8
r = 0.05
sigma = 0.4


# Initial Conditions
X0 = 20.0

# Option Parameters
T = 0.5
L1 = 15
L2 = 25

# Monte Carlo Iterations
M = 10 ** 5

## Option payoff

In [2]:
def corridor_payoff(r,T,X):
    # Take inf and sup of trajectory
    sup_X = np.max(X)
    inf_X = np.min(X)
    return np.exp(-r * T) * (sup_X <= L2 and inf_X >= L1) 

## Euler scheme with $h=0.01$

In [3]:
# Main Loop
for h in [0.01]: 
    # Fix number of step size N
    N = int(np.round(T / h))
    dt = h # Length of time interval
    
    # Store corridor option price samples with list
    corridor_sample = list()
    
    for _ in range(M):
        # Each Monte Carlo iteration
        # Build up increments of W
        W_incr = np.sqrt(dt) * np.random.randn(N + 1)
        
        # Simulation
        X = np.zeros(N + 1)
        X[0] = X0
        
        for i in range(1,N + 1):
            # Euler scheme
            X[i] = X[i - 1] * (1 + r * dt) + sigma * (X[i - 1] ** gamma) * W_incr[i - 1]
        
        # Collect samples of option price
        corridor_sample.append(corridor_payoff(r,T,X))
        
    # Take average as estimate
    corridor_est = np.mean(corridor_sample)
    
    # Compute CI
    # Compute sample standard deviation
    corridor_stddev = np.std(corridor_sample)
    
    # 95% CI
    corridor_CI_95_lb = corridor_est - 1.96 * (corridor_stddev / np.sqrt(M))
    corridor_CI_95_ub = corridor_est + 1.96 * (corridor_stddev / np.sqrt(M))
    
    # Report the results
    EM_variance = corridor_stddev ** 2
    print('When step size is ' + str(h) + ':')
    print('The 95% confidence interval for the price of corridor option is: [' + str(corridor_CI_95_lb) + ' ,' 
          + str(corridor_CI_95_ub) + '].')
    print('The variance of estimator under Euler scheme is ' + str(EM_variance))
    print(' ')

When step size is 0.01:
The 95% confidence interval for the price of corridor option is: [0.7857380356443701 ,0.7904993255811].
The variance of estimator under Euler scheme is 0.14752890632553411
 


## Richardson-Romberg Extrapolation

In [4]:
# Main Loop
for h in [0.01,0.005,0.0025]: 
    # Fix number of step size N
    N = int(np.round(T / h))
    
    # Store corridor option price samples with list
    corridor_sample = list()
    lst_fine = list()
    lst_coarse = list()
    
    for _ in range(M):
        # Each Monte Carlo iteration
        # Build up Xi as i.i.d. standard Gaussian increments
        Xi = np.random.randn(2 * N + 1)
        
        # Simulation, keep both fine and coarse scheme
        X_fine = np.zeros(2 * N + 1)
        X_fine[0] = X0
        X_coarse = np.zeros(N + 1)
        X_coarse[0] = X0
        
        # Finer scheme
        for i in range(1,2 * N + 1):
            # Euler scheme
            X_fine[i] = X_fine[i - 1] + r * X_fine[i - 1] * h / 2 + sigma * (X_fine[i - 1] ** gamma) * np.sqrt(h/2) * Xi[i - 1]
        
        # Coarser scheme
        for i in range(1,N + 1):
            # Euler scheme
            X_coarse[i] = X_coarse[i - 1] + r * X_coarse[i - 1] * h + sigma * (X_coarse[i - 1] ** gamma) * np.sqrt(h) * (Xi[2 * i - 2] + Xi[2 * i - 1]) / np.sqrt(2)
        
        #print(X_fine)
        # Collect samples of option price
        corridor_sample.append(2 * corridor_payoff(r,T,X_fine) - corridor_payoff(r,T,X_coarse)) # RR estimator!
        lst_fine.append(corridor_payoff(r,T,X_fine))
        lst_coarse.append(corridor_payoff(r,T,X_coarse))
        
    # Take average as estimate
    corridor_est = np.mean(corridor_sample)
    
    # Compute CI
    # Compute sample standard deviation
    corridor_stddev = np.std(corridor_sample)
    
    # 95% CI
    corridor_CI_95_lb = corridor_est - 1.96 * (corridor_stddev / np.sqrt(M))
    corridor_CI_95_ub = corridor_est + 1.96 * (corridor_stddev / np.sqrt(M))
    
    # Report the results
    RR_variance = corridor_stddev ** 2
    print('When step size is ' + str(h) + ':')
    print('The 95% confidence interval for the price of corridor option is: [' + str(corridor_CI_95_lb) + ' ,' 
          + str(corridor_CI_95_ub) + '].')
    print('The variance of estimator under RR scheme is ' + str(RR_variance))
    print(' ')

When step size is 0.01:
The 95% confidence interval for the price of corridor option is: [0.767320972300265 ,0.7725373292065485].
The variance of estimator under RR scheme is 0.1770771252455462
 
When step size is 0.005:
The 95% confidence interval for the price of corridor option is: [0.7647545903853765 ,0.7699150623894462].
The variance of estimator under RR scheme is 0.17330325453448392
 
When step size is 0.0025:
The 95% confidence interval for the price of corridor option is: [0.7608398749412306 ,0.765988286140884].
The variance of estimator under RR scheme is 0.17249412927370925
 


## Antithetic Sampling

In [6]:
# Main Loop
for h in [0.01]: 
    # Fix number of step size N
    N = int(np.round(T / h))
    dt = h # Length of time interval
    
    # Store corridor option price samples with list
    corridor_sample = list()
    
    # Build up increments of W in advance, note that we only build up half number of
    # BM trajectory in antithetic sampling
    W_incr = np.sqrt(dt) * np.random.randn(N + 1,int(np.ceil(M / 2)))
    
    for MC_iter in range(M):
        # Each Monte Carlo iteration
        
        # Simulation
        X = np.zeros(N + 1)
        X[0] = X0
        
        # Which trajectory to use depends on whether MC_iter is odd or even
        # If it's even, use a new trajectory
        # If it's odd, use a reflected old trajectory
        
        if MC_iter % 2 == 0:
            antithetic_traj = W_incr[:,int(np.round(MC_iter / 2))]
        else:
            antithetic_traj = -W_incr[:,int(np.round((MC_iter - 1) / 2))]
        
        for i in range(1,N + 1):
            # Euler scheme
            X[i] = X[i - 1] * (1 + r * dt) + sigma * (X[i - 1] ** gamma) * antithetic_traj[i - 1]
        
        # Collect samples of option price
        corridor_sample.append(corridor_payoff(r,T,X))
        
    # Take average as estimate
    corridor_est = np.mean(corridor_sample)
    
    # Compute CI
    # Compute sample standard deviation
    # Notice that it shall be calculated in a paired way!!!
    corridor_paired_sample = [(corridor_sample[i] + corridor_sample[i + 1]) / 2 for i in range(0,M,2)]
    corridor_stddev = np.std(corridor_paired_sample)
    
    # 95% CI
    corridor_CI_95_lb = corridor_est - 1.96 * (corridor_stddev / np.sqrt(M))
    corridor_CI_95_ub = corridor_est + 1.96 * (corridor_stddev / np.sqrt(M))
    
    # Report the results
    AT_variance = corridor_stddev ** 2
    print('When step size is ' + str(h) + ':')
    print('The 95% confidence interval for the price of corridor option is: [' + str(corridor_CI_95_lb) + ' ,' 
          + str(corridor_CI_95_ub) + '].')
    print('The variance of estimator under antithetic sampling Euler scheme is ' + str(AT_variance))
    print(' ')

When step size is 0.01:
The 95% confidence interval for the price of corridor option is: [0.7879672029463612 ,0.7919568297465759].
The variance of estimator under antithetic sampling Euler scheme is 0.10358393641315325
 
